In [ ]:

current_working_directory = '/drive/MyDrive/Colab Notebooks/'

In [ ]:
import os
def check_if_colab_environment():
    try:
        import google.colab
        return True
    except ImportError:
        return False


if check_if_colab_environment() is True:
    from google.colab import drive
    drive.mount('/drive')


if current_working_directory.endswith('/'):
    current_working_directory = current_working_directory[:-1]

os.chdir(current_working_directory)


Mounted at /drive


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd
from datasets import Dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = pd.read_csv('/drive/MyDrive/Colab Notebooks/makeup_fine_turn_data.csv')
display(dataset)
dataset = Dataset.from_pandas(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)


,input,output,instruction
0,生成一篇有關MakeUp看板心得貼文\n\n標題: integrate超水潤無瑕粉底精華沒讓...,有圖有文有素顏好讀版 <image> 雖然cat大有分享照片了 我還是想來推一下持久度XD ...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
1,生成一篇有關MakeUp看板心得貼文\n\n標題: 萊雅氣墊唇筆-酒紅色的微醺\n\n生成的...,是這樣的 在fb上看了小s的唇妝覺得唇醺的非常美 如下 <image> <image> 然後...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
2,生成一篇有關MakeUp看板心得貼文\n\n標題: nars裸光奇肌粉餅+惹火唇膏!很滿意!...,靠櫃買了版上評價還不錯的裸光奇肌粉餅 實際用在我臉上分享一下 <image> 很遮毛孔阿沒有...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
3,生成一篇有關MakeUp看板心得貼文\n\n標題: 過年就要紅吱吱唇膏分享\n\n生成的貼文...,每次回鄉過年都會特別帶比較紅的唇膏回去 土土色系長輩們都不懂( ) 分享我手邊紅得很美的顏色...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
4,生成一篇有關MakeUp看板心得貼文\n\n標題: 歐美專櫃質感 essence曼谷盤\n\...,<image> 近期用到最喜歡的開架眼影盤 essence 悠活曼谷經典眼彩盤 <image...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
...,...,...,...
4995,生成一篇有關MakeUp看板心得貼文\n\n標題: too cool for school布...,前陣子到韓國買了好多底妝品這次先跟大家分享一款 之後還會有其他彩妝品的使用報告喔 好讀圖文網...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
4996,生成一篇有關MakeUp看板心得貼文\n\n標題: [Face] 幫你遮光光！心機MAQui...,再不寫這篇2011春夏新底妝都要出了 (雖然似乎已經有形象照出來了...) (不過還是可以當...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
4997,生成一篇有關MakeUp看板心得貼文\n\n標題: 超神奇的大創粉撲清潔液\n\n生成的貼文...,網誌好讀版： <image> 這個產品在版上已經燒很久了~也有點小過時了(汗... 可是我上...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."
4998,生成一篇有關MakeUp看板心得貼文\n\n標題: NARS特霧唇膏Mascate●紅唇●c...,之前跟大家分享了一篇紅唇的文 ([妝容] NARS特霧唇膏Mascate冽豔紅唇 <imag...,"你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用""～"" ,""..."


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction', 'text'],
    num_rows: 5000
})


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,

        num_train_epochs = 5, # Set this for 1 full training run.


        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")


In [ ]:
# load model from local
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./lora_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model)

In [ ]:


inputs = tokenizer(
[
    alpaca_prompt.format(
        '''你是一個reddit 討論區的使用者, 在看板MakeUp看表心得貼文, 你愛用"～" ,"！！","><""(表達開心),"啦","XD"(表達開心)  來當語氣助詞''', # instruction
    '''生成一篇有關MakeUp看板心得貼文

標題: 媚比琳Sky High飛天翹防水睫毛膏

生成的貼文長度約: 300字

說明:
輕鬆親切：請用輕鬆親切的語氣來描述產品，像是在和朋友分享心得。
指示: "用一個輕鬆親切的語氣來撰寫心得，就像在和朋友聊天一樣。"
誠懇真實：請強調真實使用感受，避免過度誇張，讓讀者感覺到誠懇。
指示: "用誠懇真實的語氣，真誠地描述產品的優點和缺點。"
幽默風趣：適度加入一些幽默或有趣的比喻，使文章更具吸引力。
指示: "在撰寫時加入一些幽默風趣的元素，讓讀者感覺輕鬆愉快

跳過轉折詞彙，直接描述即可。
跳過叫使用者行動的字，直接描述即可。

1. 產品介紹
2. 化妝過程：應用氣墊後的妝前妝後對比，效果評價，妝感呈現。
3. 持久度測試：實際使用狀況，遇到汗水、飲食、擦臉等情況後的狀況。
4. 總結評價：整體表現評分、等總體印象。
5. 建議用途：推薦在何種情況下使用
6. 個人心得：作者的真實試用感受，以及對產品的總體滿意度和未來使用打算。

''', # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 600)

In [ ]:
# Reference here: https://github.com/unslothai/unsloth